In [3]:
from google.auth import default

credentials, project = default()

Config file has been modified. Reloading...


In [5]:
%load_ext autoreload
%autoreload 2
import os
import sys
from tqdm import tqdm

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.convert import clean_filename
from src.gcs_storage import read_from_gcs, upload_to_gcs, get_translated_phrases_path
from src.config_loader import config

print(config.TARGET_LANGUAGE_NAME)
COLLECTION = "WarmUp150"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
French


### Translations

In [6]:
# get the LM1000.json() file
from src.gcs_storage import get_story_collection_path

story_collection = read_from_gcs(
    bucket_name=config.GCS_PRIVATE_BUCKET,
    file_path=get_story_collection_path(collection=COLLECTION),
)
translated_phrases_path = get_translated_phrases_path(collection=COLLECTION)

We now want a dictionary where the key is the hash of the phrase via clean_filename -> to make an efficient lookup

In [7]:
from src.translation import translate_phrases

language_name_lower = config.TARGET_LANGUAGE_NAME.lower()

results = dict()
for story in tqdm(story_collection, desc="translating stories"):
    # Extract just the phrases from the story's phrase list
    english_phrases = [item["phrase"] for item in story_collection[story]]
    translated_phrases = translate_phrases(english_phrases)
    for phrase, translation in translated_phrases:
        phrase_key = clean_filename(phrase)
        results[phrase_key] = {"english": phrase, language_name_lower: translation}

translating stories: 100%|██████████| 8/8 [00:12<00:00,  1.55s/it]


In [8]:
results

{'i_cant_figure_it_out': {'english': "I can't figure it out",
  'french': "Je n'arrive pas à comprendre"},
 'do_you_know_the_way': {'english': 'Do you know the way?',
  'french': 'Connaissez-vous le chemin ?'},
 'lets_have_tea_this_afternoon': {'english': "Let's have tea this afternoon",
  'french': 'Prenons le thé cet après-midi'},
 'im_trying_to_lose_weight': {'english': "I'm trying to lose weight",
  'french': "J'essaie de perdre du poids"},
 'i_might_get_a_new_job': {'english': 'I might get a new job',
  'french': 'Je pourrais peut-être trouver un nouvel emploi'},
 'look_at_that_beautiful_sunset': {'english': 'Look at that beautiful sunset',
  'french': 'Regarde ce magnifique coucher de soleil'},
 'i_cant_find_my_keys': {'english': "I can't find my keys",
  'french': 'Je ne trouve pas mes clés'},
 'can_you_lift_this_box': {'english': 'Can you lift this box?',
  'french': 'Peux-tu soulever cette boîte ?'},
 'follow_the_road_straight_ahead': {'english': 'Follow the road straight ahea

In [9]:
upload_to_gcs(
    results, bucket_name=config.GCS_PRIVATE_BUCKET, file_name=translated_phrases_path
)

'gs://audio-language-trainer-private-content/collections/WarmUp150/french/translations.json'

## How to get a translation from the dictionary

In [ ]:
translation_lookup = results

example_phrase = story_collection["story_a_fishing_trip"][5]

translation_lookup[clean_filename(example_phrase)]